# Data Cleaning for African Startup Deals Data

Building a model that predicts the size of funding rounds by African tech startups based on a set of selected factors.

__Data__: [Africa: The Big Deal Startup Deals Database, March 2022 release by Max Cuvellier and Maxime Bayen.](https://thebigdeal.gumroad.com/membership)

__Note:__ For 2020, the database only captures deals of USD 500,000 or more for 2020 and deals of over USD 1 million for 2019.




### Loading data for cleaning

In [4]:
import pandas as pd
import numpy as np

In [5]:
# loading the data from the specified sheet of the XLSX file
deals = pd.read_excel("final project/2019-2022 Africa The Big Deal_Database_2022041.xlsx", 
                   sheet_name='Deals')

In [6]:
# looks like the first two rows of the spreadsheet were empty
deals.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,AFRICA STARTUPS DEALS DATABASE - 2019-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Deal Year,Deal Date,Start-up name,Website,Country HQ,Launch,Tweet pitch,Sector,# of Founders,...,Founders gender mix,Amount disclosure,Amount raised $M,Comment,Bracket,Type,Valuation $M,1st $M ?,Link to news,Investors
3,NaN,2019,2019-01-01 00:00:00,Andela,https://andela.com/,Nigeria,2014,Software developer training and outsourcing co...,Education & Jobs,4,...,Gender-diverse founding team,NaN,100,NaN,$100M+,Series D,NaN,No,https://qz.com/africa/1531075/andela-raises-10...,"Generation Investment Management, Chan Zuckerb..."
4,NaN,2019,2019-01-01 00:00:00,Bbox,https://www.bboxx.co.uk/,Africa,2010,Plug & play systems to power off-grid areas,Energy & Water,3,...,Male-only founding team,NaN,31,NaN,$10M-$50M,Venture Round,NaN,No,https://mercomindia.com/aiim-invest-31-million...,Africa Infrastructure Investment Managers


In [7]:
# making row 2 the column headers
deals.columns = deals.loc[2]

In [8]:
# dropping the empty columns and resetting the index
deals = deals.iloc[3:, 1:].reset_index(drop = True)

In [9]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1463 entries, 0 to 1462
Data columns (total 31 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   Deal Year                                    1463 non-null   object
 1   Deal Date                                    1463 non-null   object
 2   Start-up name                                1463 non-null   object
 3   Website                                      1463 non-null   object
 4   Country HQ                                   1463 non-null   object
 5   Launch                                       1463 non-null   object
 6   Tweet pitch                                  1463 non-null   object
 7   Sector                                       1462 non-null   object
 8   # of Founders                                1463 non-null   object
 9   Founder 1 (CEO)                              1452 non-null   object
 10  Founder 2   

In [10]:
# for comparison
comp = deals["Bracket"].value_counts(normalize = True)

In [11]:
# counting number of missing values for each column
deals.isnull().sum()

2
Deal Year                                         0
Deal Date                                         0
Start-up name                                     0
Website                                           0
Country HQ                                        0
Launch                                            0
Tweet pitch                                       0
Sector                                            1
# of Founders                                     0
Founder 1 (CEO)                                  11
Founder 2                                       374
Founder 3                                      1063
Founder 4                                      1365
CEO - University/School                           1
CEO - University/School - Country                 1
CEO - University/School - Continent               0
CEO Graduation year                               1
Years between graduation and startup launch       1
Woman co-founder                                 12
Woman CEO 

### Fixing dates

In [12]:
# formatting dates
deals['Deal Date'] = pd.to_datetime(deals['Deal Date']).dt.strftime('%Y-%m')
deals['Deal Month'] = pd.DatetimeIndex(deals['Deal Date']).month
deals['Deal Year'] = pd.DatetimeIndex(deals['Deal Date']).year # overwriting the year column just in case

In [13]:
# reordering columns
cols = deals.columns.tolist()
cols = cols[-1:] + cols[:-1] # last column + everything but the last column
deals = deals[cols]

### Correcting categorical levels + dropping redacted deals

In [14]:
# note that there are two fintech categories
deals["Sector"].value_counts()

Fintech                           423
Logistics & Transport             155
Energy & Water                    150
Healthcare                        129
Services                          125
Agriculture & Food                120
Retail                            119
Education & Jobs                   89
Telecom, Media & Entertainment     52
Deeptech                           44
Housing                            22
Waste Management                   16
FinTech                            15
DeepTech                            3
Name: Sector, dtype: int64

In [15]:
# combining the two fintech labels and the deeptech labels
deals["Sector"] = np.where(deals["Sector"] == "FinTech", "Fintech", deals["Sector"])
deals["Sector"] = np.where(deals["Sector"] == "DeepTech", "Deeptech", deals["Sector"])

In [16]:
# getting the index of the missing sector category for Welnes and relabelling it as 'healthcare'
location = deals["Sector"].isnull().idxmax() 
deals["Sector"][location] = "Healthcare"

/var/folders/bh/d1qm8bns06vdmp0vfb623dw40000gn/T/ipykernel_730/1999242870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deals["Sector"][location] = "Healthcare"


In [17]:
# dropping rows with redacted deal info 
excl = list(deals[deals['Start-up name'] == "Confidential"].index)
deals = deals.drop(deals.index[excl]).reset_index(drop = True)

In [18]:
# counting number of null values
deals.isnull().sum()

2
Deal Month                                        0
Deal Year                                         0
Deal Date                                         0
Start-up name                                     0
Website                                           0
Country HQ                                        0
Launch                                            0
Tweet pitch                                       0
Sector                                            0
# of Founders                                     0
Founder 1 (CEO)                                   0
Founder 2                                       359
Founder 3                                      1037
Founder 4                                      1335
CEO - University/School                           1
CEO - University/School - Country                 1
CEO - University/School - Continent               0
CEO Graduation year                               1
Years between graduation and startup launch       1
Woman co-f

In [19]:
# missing values for University and University - Country are for the same deal
deals["CEO - University/School"].isnull().idxmax() == deals["CEO - University/School - Country"].isnull().idxmax()

True

In [20]:
# entering in information scraped from LinkedIn to fill empty value
location = deals["CEO - University/School"].isnull().idxmax()
deals["CEO - University/School"][location] = "Federal University Of Agriculture, Abeokuta"
deals["CEO - University/School - Country"][location] = "Nigeria"
deals["CEO - University/School - Continent"][location] = "Africa"

/var/folders/bh/d1qm8bns06vdmp0vfb623dw40000gn/T/ipykernel_730/3671636214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deals["CEO - University/School"][location] = "Federal University Of Agriculture, Abeokuta"
/var/folders/bh/d1qm8bns06vdmp0vfb623dw40000gn/T/ipykernel_730/3671636214.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deals["CEO - University/School - Country"][location] = "Nigeria"
/var/folders/bh/d1qm8bns06vdmp0vfb623dw40000gn/T/ipykernel_730/3671636214.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

### Dropping NA values + changing variable type

In [21]:
# getting rid of n.a. strings in the two fields
excl_1 = list(np.unique(list(deals[deals["Woman co-founder"] == 'n.a'].index) + list(deals[deals["Amount raised $M"] == 'n.a'].index)))
deals = deals.drop(deals.index[excl_1]).reset_index(drop = True)

In [22]:
# making data fields numeric
deals['Amount raised $M'] = pd.to_numeric(deals['Amount raised $M'])
deals['Launch'] = pd.to_numeric(deals['Launch'])
deals['# of Founders'] = pd.to_numeric(deals['# of Founders'])

In [23]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393 entries, 0 to 1392
Data columns (total 32 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Deal Month                                   1393 non-null   int64  
 1   Deal Year                                    1393 non-null   int64  
 2   Deal Date                                    1393 non-null   object 
 3   Start-up name                                1393 non-null   object 
 4   Website                                      1393 non-null   object 
 5   Country HQ                                   1393 non-null   object 
 6   Launch                                       1393 non-null   int64  
 7   Tweet pitch                                  1393 non-null   object 
 8   Sector                                       1393 non-null   object 
 9   # of Founders                                1393 non-null   int64  
 10  

In [24]:
# filling in categories using data from founders gender mix
deals['Women only founders'] = np.where((deals["Founders gender mix"] == "Female single founder") | 
                                        (deals["Founders gender mix"] == "Female-only founding team") &
                                        (deals['Women only founders'] == ''), "Women only founders",
                                        deals['Women only founders'])

In [25]:
# filling in NA values as no
deals['Women only founders'] = np.where((deals["Women only founders"] != 'No') & 
                                        (deals["Women only founders"] != 'Women only founders'), 
                                        'No', deals["Women only founders"])

deals['Woman CEO'] = np.where((deals["Woman CEO"] != 'No') & 
                                        (deals["Woman CEO"] != 'Woman CEO'), 
                                        'No', deals["Woman CEO"])

In [26]:
# getting rid of extraneous levels in the gender mix variable
deals['Founders gender mix'] = np.where((deals["Founders gender mix"] == 'Male-only founding team') | 
                                        (deals["Founders gender mix"] == 'Male single founder'),
                                       'Male-only founding team', deals['Founders gender mix'])

deals['Founders gender mix'] = np.where((deals["Founders gender mix"] == 'Female single founder') | 
                                        (deals["Founders gender mix"] == 'Female-only founding team'),
                                       'Female-only founding team', deals['Founders gender mix'])

In [27]:
# startups with Country HQ listed as Africa are based overseas
np.unique(deals[deals['Country HQ'] == 'Africa']['Start-up name'])

# changing Africa category in Country HQ to Overseas
deals['Country HQ'] = np.where((deals['Country HQ'] == 'Africa'), 'Overseas', deals['Country HQ'])

In [28]:
# fixing category levels in type variable
deals['Type'] = np.where((deals['Type'] == 'Venture round'), 'Venture Round', deals['Type'])
deals['Type'] = np.where((deals['Type'] == 'Pre-seed'), 'Pre-Seed', deals['Type'])
deals['Type'] = np.where((deals['Type'] == 'Pre-series A'), 'Pre-Series A', deals['Type'])
deals['Type'] = np.where((deals['Type'] == 'Pre-series B'), 'Pre-Series B', deals['Type'])

In [29]:
# dropping rows with n.a in CEO - University/School - Continent
excl_2 = list(deals[deals['CEO - University/School - Continent'] == 'n.a'].index)
deals = deals.drop(deals.index[excl_2]).reset_index(drop = True)

In [30]:
# dropping rows with n.a in Years between graduation and startup launch
excl_3 = list(deals[deals['Years between graduation and startup launch'] == 'n.a'].index)
deals = deals.drop(deals.index[excl_3]).reset_index(drop = True)

In [31]:
# deal brackets after data cleaning
deals["Bracket"].value_counts(normalize = True)

$100K-$500K    0.348739
$1M-$2M        0.180672
$2M-$5M        0.145378
$500K-$1M      0.128571
$10M-$50M      0.101681
$5M-$10M       0.069748
$50M-$100M     0.013445
$100M+         0.011765
Name: Bracket, dtype: float64

In [32]:
# deal brackets before data cleaning
# not a huge difference between the two
comp

$100K-$500K    0.345865
$1M-$2M        0.173616
$2M-$5M        0.142174
$500K-$1M      0.129187
$10M-$50M      0.096377
$5M-$10M       0.064252
n.a            0.020506
$100M+         0.014354
$50M-$100M     0.013671
Name: Bracket, dtype: float64

In [33]:
# write file to excel
deals.to_excel("deals.xlsx")

In [34]:
# writing dataset that removes outliers
sd = np.std(deals['Amount raised $M'])
mean = np.mean(deals['Amount raised $M'])

upper_bound = mean + (3 * sd) # filtering out obs 3 s.d. from the mean
lower_bound = mean - (3 * sd)

deals_clean = deals[(deals['Amount raised $M'] > lower_bound) & (deals['Amount raised $M'] < upper_bound)].reset_index()

# write file to excel
deals_clean.to_excel("deals_clean.xlsx")

In [35]:
deals_clean.shape

(1167, 33)

In [36]:
deals.shape

(1190, 32)

In [37]:
deals_clean.describe()

2,index,Deal Month,Deal Year,Launch,# of Founders,Amount raised $M
count,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000,1167.000000
mean,593.379606,6.155955,2020.823479,2016.828620,2.109683,3.746015
std,343.397050,3.637476,0.809271,3.205073,0.846727,8.041556
min,1.000000,1.000000,2019.000000,2000.000000,1.000000,0.100000
25%,296.500000,3.000000,2020.000000,2015.000000,2.000000,0.200000
50%,592.000000,6.000000,2021.000000,2018.000000,2.000000,1.000000
75%,891.500000,10.000000,2021.000000,2019.000000,3.000000,3.000000
max,1189.000000,12.000000,2022.000000,2022.000000,4.000000,65.000000


In [38]:
deals.describe()

2,Deal Month,Deal Year,Launch,# of Founders,Amount raised $M
count,1190.000000,1190.000000,1190.000000,1190.000000,1190.000000
mean,6.157983,2020.825210,2016.744538,2.119328,6.108908
std,3.640523,0.809762,3.284177,0.849011,20.460387
min,1.000000,2019.000000,2000.000000,1.000000,0.100000
25%,3.000000,2020.250000,2015.000000,2.000000,0.200000
50%,6.000000,2021.000000,2017.000000,2.000000,1.000000
75%,10.000000,2021.000000,2019.000000,3.000000,3.200000
max,12.000000,2022.000000,2022.000000,4.000000,288.000000
